## DATA WRANGLING - @WERATEDOGS DATASET
In this project,

In [6]:
# import installed packages
import pandas as pd
import numpy as np
import requests
import json
import tweepy
from tweepy import OAuthHandler
from timeit import default_timer as timer

In [85]:
# setup tweepy
_consumer_key = ''
_consumer_secret = ''
_access_token = ""
_access_token_secret = ""

with open('auth_keys.txt', 'r') as auth_keys:
    try:
        _consumer_key = auth_keys.readline().split('"')[1:-1][0]
        _consumer_secret = auth_keys.readline().split('"')[1:-1][0]
        _access_token = auth_keys.readline().split('"')[1:-1][0]
        _access_token_secret = auth_keys.readline().split('"')[1:-1][0]
    except:
        raise Exception('Error: auth_keys.txt is missing or keys not found in source.')

auth = OAuthHandler(_consumer_key, _consumer_secret)
auth.set_access_token(_access_token, _access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [3]:
# reusable functions
def get_image_predictions_tsv():
    url = " https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
    response = requests.get(url)
    with open('image-predictions.tsv', 'wb') as file:
        file.write(response.content)
        
def get_tweet_status(tweet_id):
    if (isinstance(tweet_id, str)):
        try:
            tweet_status = api.get_status(tweet_id, tweet_mode='extended')
            return tweet_status # retweet_count | favorite_count | id_str
        except:
            return None
    else:
        raise Exception("Invalid argument types passed to function.")      

In [87]:
# gather data
tweets_df = pd.read_csv("twitter-archive-enhanced.csv", index_col=False)
images_df = pd.read_csv("image-predictions.tsv", sep="\t", index_col="tweet_id")

In [ ]:
# gather each tweet's retweet_count and favorite_count
with open('tweet_json.txt', 'w') as file:
        start = timer()
        for index, row in tweets_df.iterrows():
            tweet_status = get_tweet_status(str(row['tweet_id']))
            if tweet_status is None:
                file.write('id_str: u\'Invalid\',retweet_count\': Invalid, favorite_count\': Invalid,\n')
            file.write(str(tweet_status) + '\n')
        end = timer()

print('Time taken: ' + end - start)
    

In [95]:
df_data = {'tweet_id': [], 'retweet_count': [], 'favorite_count': []}

with open('tweet_json.txt', 'r') as tweet_json:
    lines = tweet_json.read().split('\n');
    for line in lines:
        try:
           df_data['tweet_id'].append(line.split('id_str\': u\'')[1].split('\'')[0])
           df_data['retweet_count'].append(line.split('retweet_count\': ')[1].split(',')[0])
           df_data['favorite_count'].append(line.split('favorite_count\': ')[1].split(',')[0]) 
        except:
            pass

tweets_meta_df = pd.DataFrame(df_data)